**Библиотеки**

In [47]:
import logging
import sys
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Callable, Tuple, List, Optional

import numpy as np
import sympy as sp
import jax.numpy as jnp
from jax import jacfwd
from sympy import Eq, Rel

In [48]:
@dataclass
class OptimizationConfig:
    """Configuration for the optimization algorithm."""
    max_iterations: int = 1000
    input_file_path: Optional[str] = None
    log_to_file: bool = True

In [49]:
@dataclass
class OptimizationProblem:
    """Represents an optimization problem with constraints."""
    target_function: Callable
    target_expression: sp.Expr
    x_symbols: List[sp.Symbol]
    equalities: List[sp.Expr]
    inequalities: List[sp.Expr]
    lambda_symbols: List[sp.Symbol]
    nu_symbols: List[sp.Symbol]
    s_symbols: List[sp.Symbol]

In [50]:
@dataclass
class InitialPoint:
    """Initial values for optimization variables."""
    x: np.ndarray
    s: np.ndarray
    lambdas: np.ndarray
    mu: float
    epsilon: float

In [51]:
def setup_logger(log_to_file: bool) -> logging.Logger:
    """
    Configure logger with console and optional file output.
    
    Args:
        log_to_file: Whether to save logs to a file
        
    Returns:
        Configured logger instance
    """
    logger = logging.getLogger("interior_point_optimizer")
    logger.setLevel(logging.INFO)
    logger.handlers.clear()
    logger.propagate = False

    # Console handler
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(logging.Formatter("%(message)s"))
    logger.addHandler(console_handler)

    # File handler
    if log_to_file:
        logs_dir = Path("logs")
        logs_dir.mkdir(exist_ok=True)
        
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        file_path = logs_dir / f"optimization_log_{timestamp}.txt"
        
        file_handler = logging.FileHandler(file_path, encoding="utf-8")
        file_handler.setFormatter(logging.Formatter("%(message)s"))
        logger.addHandler(file_handler)

    return logger

In [52]:
def parse_constraint(constraint_string: str) -> Tuple[Optional[str], Optional[sp.Expr]]:
    """
    Parse a constraint string into its type and expression.
    
    Args:
        constraint_string: String representation of the constraint
        
    Returns:
        Tuple of (constraint_type, expression) where type is 'equality' or 'inequality'
    """
    constraint_string = constraint_string.strip()

    try:
        # Handle equality constraints
        if "=" in constraint_string and all(op not in constraint_string for op in [">=", "<=", "!="]):
            constraint_string = constraint_string.replace("==", "=")
            left, right = constraint_string.split("=", 1)
            lhs = sp.sympify(left)
            rhs = sp.sympify(right)
            return "equality", lhs - rhs
        
        # Handle inequality constraints
        expr = sp.sympify(constraint_string)
        
        # Unwrap negations of equalities
        while isinstance(expr, sp.Not) and isinstance(expr.args[0], sp.Eq):
            expr = expr.args[0]

        if isinstance(expr, Eq):
            return "equality", expr.lhs - expr.rhs
        elif isinstance(expr, Rel):
            return "inequality", expr.lhs - expr.rhs
        
        return None, None
        
    except Exception as e:
        logging.error(f"Error parsing constraint '{constraint_string}': {e}")
        return None, None

In [53]:
def read_problem_from_file(file_path: Path) -> Tuple[OptimizationProblem, InitialPoint]:
    """
    Read optimization problem and initial values from file.
    
    Args:
        file_path: Path to input file
        
    Returns:
        Tuple of (OptimizationProblem, InitialPoint)
    """
    if not file_path.exists():
        raise FileNotFoundError(f"File '{file_path}' does not exist!")

    x, s, lambdas, mu, epsilon = None, None, None, None, None
    equalities, inequalities = [], []
    target_expr = None
    in_constraints_section = False

    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            
            # Skip empty lines and comments
            if not line or line.startswith("#"):
                continue

            # Check for constraints section
            if line.lower().startswith("constraints"):
                in_constraints_section = True
                continue

            # Parse constraints
            if in_constraints_section:
                constraint_type, expr = parse_constraint(line)
                if constraint_type == "equality":
                    equalities.append(expr)
                elif constraint_type == "inequality":
                    inequalities.append(expr)
                continue

            # Parse key-value pairs
            if ":" not in line:
                continue
                
            key, value = line.split(":", 1)
            key = key.strip()
            value = value.strip()

            if key == "target_function":
                target_expr = sp.sympify(value)
            elif key == "x":
                x = np.array([float(v) for v in value.split()])
            elif key == "s":
                s = np.array([float(v) for v in value.split()])
            elif key == "lambdas":
                lambdas = np.array([float(v) for v in value.split()])
            elif key == "mu":
                mu = float(value)
            elif key in ("epsilon", "eps"):
                epsilon = float(value)

    # Create symbolic variables
    x_symbols = sorted(target_expr.free_symbols, key=lambda s: s.name)
    lambda_symbols = sp.symbols(f'λ1:{len(equalities) + 1}')
    nu_symbols = sp.symbols(f'ν1:{len(inequalities) + 1}')
    s_symbols = sp.symbols(f"s1:{len(inequalities) + 1}")

    # Create problem and initial point
    problem = OptimizationProblem(
        target_function=sp.lambdify(x_symbols, target_expr),
        target_expression=target_expr,
        x_symbols=x_symbols,
        equalities=equalities,
        inequalities=inequalities,
        lambda_symbols=list(lambda_symbols),
        nu_symbols=list(nu_symbols),
        s_symbols=list(s_symbols)
    )

    initial = InitialPoint(x=x, s=s, lambdas=lambdas, mu=mu, epsilon=epsilon)

    return problem, initial

In [54]:
def create_lagrangian(
    problem: OptimizationProblem,
    mu: float
) -> sp.Expr:
    """
    Construct the Lagrangian with barrier terms.
    
    Args:
        problem: Optimization problem specification
        mu: Barrier parameter
        
    Returns:
        Symbolic Lagrangian expression
    """
    L = problem.target_expression
    
    # Add barrier terms for slack variables
    L -= mu * sum(sp.log(s) for s in problem.s_symbols)
    
    # Add equality constraints
    for lam, h in zip(problem.lambda_symbols, problem.equalities):
        L += lam * h
    
    # Add inequality constraints with slack variables
    for nu, g, s in zip(problem.nu_symbols, problem.inequalities, problem.s_symbols):
        L += nu * (g + s)
    
    # Substitute ν = μ/s (complementarity condition)
    nu_substitution = {nu: mu / s for nu, s in zip(problem.nu_symbols, problem.s_symbols)}
    L = L.subs(nu_substitution)
    
    return L

In [55]:
def create_kkt_system(
    lagrangian: sp.Expr,
    x_symbols: List[sp.Symbol],
    s_symbols: List[sp.Symbol],
    lambda_symbols: List[sp.Symbol]
) -> Callable:
    """
    Create the KKT system of equations from the Lagrangian.
    
    Args:
        lagrangian: Lagrangian expression
        x_symbols: Decision variables
        s_symbols: Slack variables
        lambda_symbols: Lagrange multipliers
        
    Returns:
        Function that computes KKT system residuals
    """
    # Compute gradients
    grad_x = [sp.diff(lagrangian, x) for x in x_symbols]
    grad_lambda = [sp.diff(lagrangian, lam) for lam in lambda_symbols]
    grad_s = [sp.diff(lagrangian, s) for s in s_symbols]
    
    # Combine all equations
    system_equations = grad_x + grad_lambda + grad_s
    all_symbols = list(x_symbols) + list(lambda_symbols) + list(s_symbols)
    
    # Create numerical function using JAX
    system_func = sp.lambdify(all_symbols, system_equations, modules="jax")
    
    def kkt_residual(y: jnp.ndarray) -> jnp.ndarray:
        return jnp.array(system_func(*y))
    
    return kkt_residual

In [56]:
def solve_interior_point(
    initial: InitialPoint,
    kkt_system: Callable,
    max_iterations: int,
    logger: logging.Logger
) -> Tuple[np.ndarray, float, np.ndarray]:
    """
    Solve optimization problem using interior point method.
    
    Args:
        initial: Initial values for all variables
        kkt_system: Function computing KKT residuals
        max_iterations: Maximum number of iterations
        logger: Logger for progress tracking
        
    Returns:
        Tuple of (optimal_x, optimal_lambda, optimal_s)
    """
    # Combine all variables into single vector
    y = jnp.array(np.concatenate([initial.x, initial.lambdas, initial.s]))
    x_dim = len(initial.x)
    s_dim = len(initial.s)
    
    for iteration in range(max_iterations):
        # Compute KKT residuals
        F = kkt_system(y)
        residual = float(jnp.linalg.norm(F))
        
        logger.info(f'Residual: {residual}')
        
        # Check convergence
        if residual < initial.epsilon:
            logger.info(f"Converged: {residual} < {initial.epsilon}\n")
            break
        
        logger.info("")
        logger.info(f"Iteration {iteration + 1}")
        logger.info(f'F(y{iteration}) = ({", ".join(f"{v:.6g}" for v in F)})')
        
        # Compute Jacobian
        J = jacfwd(kkt_system)(y)
        logger.info(f'J(y{iteration}):\n{J}')
        
        # Solve Newton system
        delta = jnp.linalg.solve(J, -F)
        logger.info(f'\nΔ{iteration + 1} = ({", ".join(f"{v:.6g}" for v in delta)})')
        
        # Backtracking line search to maintain s > 0
        step = 1.0
        max_backtrack = 20
        
        for _ in range(max_backtrack):
            y_candidate = y + step * delta
            s_new = y_candidate[-s_dim:]
            
            if np.all(s_new > 0):
                break
            
            step *= 0.5
            logger.info(f"Slack variables would be negative. Reducing step to {step}")
        else:
            logger.warning("Could not find valid step maintaining s > 0. Stopping.")
            break
        
        # Update variables
        y = y_candidate
        logger.info(f'y{iteration + 1} = ({", ".join(f"{v:.6g}" for v in y)})')
        
        # Log new residual
        F_new = kkt_system(y)
        logger.info(f'F(y{iteration + 1}) = ({", ".join(f"{v:.6g}" for v in F_new)})\n')
    else:
        logger.info("Maximum iterations reached.")
    
    # Extract solution components
    x_opt = y[:x_dim]
    lambda_opt = y[x_dim]
    s_opt = y[x_dim + 1:]
    
    return x_opt, lambda_opt, s_opt

In [57]:
def log_initial_parameters(
    initial: InitialPoint,
    problem: OptimizationProblem,
    logger: logging.Logger
) -> None:
    """Log initial parameter values."""
    logger.info('Initial Parameters:\n')
    
    logger.info('Initial approximation x:')
    for i, val in enumerate(initial.x, 1):
        logger.info(f'x{i} = {val}')
    logger.info('')
    
    logger.info('Initial slack variables s:')
    for i, val in enumerate(initial.s, 1):
        logger.info(f's{i} = {val}')
    
    logger.info('Initial Lagrange multipliers λ:')
    for i, val in enumerate(initial.lambdas, 1):
        logger.info(f'λ{i} = {val}')
    logger.info('')
    
    logger.info(f'μ = {initial.mu}')
    logger.info(f'ε = {initial.epsilon}\n')


def log_solution(
    x_opt: np.ndarray,
    lambda_opt: float,
    s_opt: np.ndarray,
    target_function: Callable,
    logger: logging.Logger
) -> None:
    """Log the optimal solution."""
    logger.info('Optimal Solution:')
    
    for i, val in enumerate(x_opt, 1):
        logger.info(f'x{i}* = {val}')
    
    logger.info(f'λ* = {lambda_opt}')
    
    for i, val in enumerate(s_opt, 1):
        logger.info(f's{i}* = {val}')
    
    obj_value = target_function(*x_opt)
    logger.info(f'Objective function value F(x*) = {obj_value}')

In [58]:
def main():
    """Main execution function."""
    # Configuration
    config = OptimizationConfig(
        max_iterations=1000,
        input_file_path="input.txt",  # Set to filename for file input
        log_to_file=True
    )
    
    logger = setup_logger(config.log_to_file)
    
    # Load problem (implement interactive input if needed)
    if config.input_file_path:
        problem, initial = read_problem_from_file(Path(config.input_file_path))
    else:
        raise NotImplementedError("Interactive input not implemented in refactored version")
    
    # Build optimization problem
    lagrangian = create_lagrangian(problem, initial.mu)
    kkt_system = create_kkt_system(
        lagrangian,
        problem.x_symbols,
        problem.s_symbols,
        problem.lambda_symbols
    )
    
    # Log initial state
    log_initial_parameters(initial, problem, logger)
    logger.info("Newton's Method:\n")
    
    # Solve
    x_opt, lambda_opt, s_opt = solve_interior_point(
        initial,
        kkt_system,
        config.max_iterations,
        logger
    )
    
    # Log results
    log_solution(x_opt, lambda_opt, s_opt, problem.target_function, logger)


if __name__ == "__main__":
    main()

Initial Parameters:

Initial approximation x:
x1 = 1.0
x2 = 1.0
x3 = 1.0

Initial slack variables s:
s1 = 1.0
s2 = 1.0
Initial Lagrange multipliers λ:
λ1 = 0.0

μ = 0.1
ε = 0.001

Newton's Method:

Residual: 3.753664970397949

Iteration 1
F(y0) = (2.2, 2.2, 2.1, 0, -0, -0)
J(y0):
[[ 2.2  0.   0.   2.  -0.2  0. ]
 [ 0.   2.2  0.   1.  -0.2  0. ]
 [ 0.   0.   2.2  0.  -0.2  0.1]
 [ 2.   1.   0.   0.   0.   0. ]
 [-0.2 -0.2 -0.2  0.  -0.1  0. ]
 [ 0.   0.   0.1  0.   0.  -0.1]]

Δ1 = (0.166558, -0.333116, -0.753102, -1.09928, 1.83932, -0.753102)
y1 = (1.16656, 0.666884, 0.246898, -1.09928, 2.83932, 0.246898)
F(y1) = (-0.149464, 0.281462, 0.10616, 0.0277414, -0.00875581, -9.77792e-08)

Residual: 0.3371596038341522

Iteration 2
F(y1) = (-0.149464, 0.281462, 0.10616, 0.0277414, -0.00875581, -9.77792e-08)
J(y1):
[[-0.12812372  0.          0.          2.3331156  -0.02894059  0.        ]
 [ 0.          2.0704393   0.          1.         -0.01654443  0.        ]
 [ 0.          0.          2.0704